In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv' )
df.dropna(inplace  = True)
print(df)

For this notebook , I tend to use 'Open' , 'High' , 'Low' , 'Volume_(BTC)' ,'Volume_(Currency)' to predict the price of three  minutes later 

In [ ]:
from datetime import datetime
df.drop(columns = ['Weighted_Price'] , inplace = True)

df['Timestamp'] = df['Timestamp'].apply(datetime.fromtimestamp)
df.set_index('Timestamp' , inplace = True)
df.sort_index(inplace = True)


I only use 100k rows for the first trial to save running time.


In [ ]:
df = df.iloc[3500000: , :]

In [ ]:
def data_treat(df , shift_days  , window ):
    #####the labels are from minutes later
    df['target'] = df['Close'].shift(-shift_days)
    df.dropna(inplace = True)
    df
    ###
    x = df.iloc[:,:-1]
    y = df.iloc[: ,-1]
    from sklearn.preprocessing import MinMaxScaler

    ###standardlize
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)
    from collections import deque
    deq = deque(maxlen = window)

    ###transform x into input of LSTM
    train_x = []
    for i in list(x):
        deq.append(list(i))

        if len(deq)==window:
            train_x.append(list(deq))

    train_x_last = train_x[-shift_days:]
    train_x = train_x[:-shift_days]
    train_y = y.values[window-1:-shift_days]
    X = np.array(train_x)
    Y = np.array(train_y)

    from sklearn.model_selection import train_test_split
    X_train , X_test , y_train , y_test = train_test_split(X , Y , test_size = 0.1 ,shuffle = False )


    return X_train , X_test , y_train , y_test

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM ,Dense ,GRU, Dropout , BatchNormalization
from tensorflow.keras import regularizers


In [ ]:
X_train , X_test , y_train , y_test = data_treat(df , 2 , 7)

In [ ]:
def get_model(wd):
    model = Sequential(
    [
        LSTM(64 , kernel_regularizer = regularizers.l2(wd),input_shape =X_train.shape[1:] , activation = 'relu' , return_sequences = True ),
        
        Dropout(0.3),
        LSTM(64 ,kernel_regularizer = regularizers.l2(wd) , activation = 'relu' , return_sequences = True ),
        
        Dropout(0.3),
        GRU(64 ,kernel_regularizer = regularizers.l2(wd) , activation = 'relu' , return_sequences = True ),
        
        Dropout(0.2),
        GRU(64 ,kernel_regularizer = regularizers.l2(wd) , activation = 'relu' , return_sequences = True ),
        
        Dropout(0.2),
        
        
        
        LSTM(32,kernel_regularizer = regularizers.l2(wd)  , activation = 'relu' , return_sequences = False ),
        
       
        Dropout(0.3),
        
        
        
        
        Dense(1)
    ]
)
    model.compile(optimizer ='adam' ,loss = 'mse' , metrics = ['mape'])
    return model 


In [ ]:
model = get_model(1e-5)

In [ ]:
model.summary()

In [ ]:

X_train.shape[1:]


In [ ]:
def get_callbacks():

    callbacks1 = tf.keras.callbacks.EarlyStopping(patience = 30 , monitor = 'val_mape' , mode = 'min')
    callbacks2 = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.2 , patience =30)
    callback  = [callbacks1 , callbacks2]
    return callback

In [ ]:
early_stopping, learning_rate_reduction = get_callbacks()

In [ ]:
history = model.fit(X_train , y_train ,batch_size =256 ,epochs = 50 , validation_split = 0.15 , verbose = 1   , callbacks = [early_stopping , learning_rate_reduction])

In [ ]:
pred_y = model.predict(X_test)

In [ ]:
model.evaluate(X_train , y_train)

In [ ]:
import matplotlib.pyplot as plt
y_index = list(range(len(pred_y)))
plt.plot( y_index ,  pred_y  , color = 'red')

plt.plot(y_index , y_test)
plt.show()

In [ ]:
pred_y = model.predict(X_train)
y_index = list(range(len(pred_y)))
plt.plot( y_index ,  pred_y  , color = 'red')

plt.plot(y_index , y_train)
plt.show()